In [8]:
stopwords = list(set([
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', '하다', '있다',
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다',
    '고', '면', '게', '지', '죠',
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데',
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구', '그렇다',
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',
    '바로', '때', '것', '수', '문제', '경우', '부분', '이다',
    '내용', '결과', '자체', '가지', '있다',
    '않았어요', '있었어요', '했어요', '했는데요', '있는데요', '합니다', '없다', '나다',
    '했다', '같다', '네요','아니다',
    '용하다', '물이', '뿐', '대로', '만', '따름', '김에', '터',
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니', '않다', '안되다','안','그냥',
    '가다', '오다', '주다', '말다', '나다', '받다', '알다', '모르다', '싶다', '생각하다', '들다'
]))

In [ ]:
from functools import partial
from konlpy.tag import Okt
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# 2. 형태소 분석기 + tokenizer 함수 정의
okt = Okt()

def tokenize(text, stopwords=[]):
    try:
        return [
            word.lower()
            for word, pos in okt.pos(text, stem=True)
            if pos in ['Noun', 'Adjective', 'Verb']
            and word.lower() not in stopwords
            and len(word) > 1
        ]
    except Exception as e:
        print(f"Tokenization error: {e}")
        return []

In [10]:
import joblib

model = joblib.load('Logistic_model.pkl')
vectorizer = joblib.load('Logistic_tfdf_vectorizer.pkl')

In [ ]:
# 새 문장 리스트
import pandas as pd
df = pd.read_csv("../data/new_test_data.csv")

sentence = df['text']

# 벡터화 (학습한 vectorizer 사용)
X_new = vectorizer.transform(sentence)

# 예측 수행
predictions = model.predict(X_new)
probs = model.predict_proba(X_new)

labels = []
threshold = 0.6
for i, text in enumerate(sentence):
    prob_pos = probs[i][1]
    if prob_pos >= threshold:
        label = 1
    elif prob_pos <= 1 - threshold:
        label = -1
    else:
        label = 0
    labels.append(label)

df['TF_label'] = labels

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 모델 정의
logit = LogisticRegression(max_iter=1000)
tree = DecisionTreeClassifier(max_depth=5)
forest = RandomForestClassifier(n_estimators=100)

# 모델 훈련
# logit.fit(X_train, y_train)
# tree.fit(X_train, y_train)
# forest.fit(X_train, y_train)

# # 예측 및 정확도 평가
# for model in [logit, tree, forest]:
#     preds = model.predict(X_test)
#     print(model.__class__.__name__, "정확도:", accuracy_score(y_test, preds))

In [16]:
nw_df = df[['text','label','TF_label']]

In [17]:
nw_df.to_csv('TF_test_data.csv', encoding='utf-8-sig', index=False)
nw_df['TF_label'].value_counts()

TF_label
-1    6061
 1    4643
Name: count, dtype: int64

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix

from konlpy.tag import Okt
import warnings

font_path = "C:/Windows/Fonts/NanumGothic.ttf"
font_prop = fm.FontProperties(fname=font_path)
plt.rc('font', family=font_prop.get_name())
plt.rcParams['axes.unicode_minus'] = False

# 6. 중요 단어 추출 (이진 분류는 coef_[0] 사용)
feature_names = np.array(vectorizer.get_feature_names_out())
coef = model.coef_[0]

topn = 20
top_pos_idx = np.argsort(coef)[::-1][:topn]
top_neg_idx = np.argsort(coef)[:topn]

df_pos = pd.DataFrame({'word': feature_names[top_pos_idx], 'weight': coef[top_pos_idx]})
df_neg = pd.DataFrame({'word': feature_names[top_neg_idx], 'weight': coef[top_neg_idx]})

fig, axes = plt.subplots(1, 2, figsize=(18, 10), sharey=True)

sns.barplot(ax=axes[0], data=df_neg, y='word', x='weight', color='#e74c3c')
axes[0].set_title("부정 상위 단어 (label=0)")
axes[0].set_xlabel("가중치(weight)")
axes[0].set_ylabel("단어")

sns.barplot(ax=axes[1], data=df_pos, y='word', x='weight', color='#2ecc71')
axes[1].set_title("긍정 상위 단어 (label=1)")
axes[1].set_xlabel("가중치(weight)")

plt.tight_layout()
plt.show()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset
import pandas as pd
import torch

# 1. 데이터 로드
df = pd.read_csv("keyword_test.csv")  # 'clean_reviews'와 'label' 컬럼이 있어야 함
df = df.dropna(subset=['clean_reviews', 'label'])  # 결측 제거
df['label'] = df['label'].astype(int)  # 레이블 정수형 변환

# 2. 데이터셋 분할
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# 3. 토크나이저 로드 (KoBERT)
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 4. 토큰화 함수 정의
def tokenize_function(example):
    return tokenizer(example['clean_reviews'], padding="max_length", truncation=True, max_length=128)

# 5. HuggingFace Dataset으로 변환
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# 6. 토큰화 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# 7. label, input_ids, attention_mask만 추림
train_dataset = train_dataset.remove_columns(['clean_reviews'])
val_dataset = val_dataset.remove_columns(['clean_reviews'])

# 8. PyTorch tensor 형식 명시
train_dataset.set_format("torch")
val_dataset.set_format("torch")

# 9. 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 10. 평가지표 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }

# 11. 학습 설정
training_args = TrainingArguments(
    output_dir="./bert_sentiment_output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=1,
    seed=42
)

# 12. Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# 13. 학습 실행
trainer.train()

# 14. 모델 저장
trainer.save_model("./saved_bert_sentiment_model")
tokenizer.save_pretrained("./saved_bert_sentiment_model")
